# ROI Sweep Demo

This notebook demonstrates how to sweep simulations across the sensor image plane,
modeling the variation of QE with sensor position due to changing Chief Ray Angle (CRA).

Topics covered:
1. Define a CRA vs image height curve
2. Run `ROISweepRunner` at multiple sensor positions
3. Extract per-channel QE at each position
4. Plot QE vs image height
5. Compute and plot relative illumination

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from compass.runners.roi_sweep_runner import ROISweepRunner
from compass.runners.single_run import SingleRunner

## 1. Define Pixel Configuration

We use a standard 2x2 BSI pixel with microlens, color filter, BARL, and DTI.
The microlens shift mode is set to `none` initially; the `ROISweepRunner`
will override it to `auto_cra` at each position.

In [ ]:
config = {
    "pixel": {
        "pitch": 1.0,
        "unit_cell": [2, 2],
        "bayer_map": [["R", "G"], ["G", "B"]],
        "layers": {
            "air": {"thickness": 1.0, "material": "air"},
            "microlens": {
                "enabled": True, "height": 0.6,
                "radius_x": 0.48, "radius_y": 0.48,
                "material": "polymer_n1p56",
                "profile": {"type": "superellipse", "n": 2.5, "alpha": 1.0},
                "shift": {"mode": "none"},
            },
            "planarization": {"thickness": 0.3, "material": "sio2"},
            "color_filter": {
                "thickness": 0.6,
                "materials": {"R": "cf_red", "G": "cf_green", "B": "cf_blue"},
                "grid": {"enabled": True, "width": 0.05, "material": "tungsten"},
            },
            "barl": {"layers": [
                {"thickness": 0.010, "material": "sio2"},
                {"thickness": 0.025, "material": "hfo2"},
                {"thickness": 0.015, "material": "sio2"},
                {"thickness": 0.030, "material": "si3n4"},
            ]},
            "silicon": {
                "thickness": 3.0, "material": "silicon",
                "photodiode": {"position": [0, 0, 0.5], "size": [0.7, 0.7, 2.0]},
                "dti": {"enabled": True, "width": 0.1, "material": "sio2"},
            },
        },
    },
    "solver": {
        "name": "torcwa", "type": "rcwa",
        "params": {"fourier_order": [9, 9]},
        "stability": {"precision_strategy": "mixed", "fourier_factorization": "li_inverse"},
    },
    "source": {
        "wavelength": {"mode": "single", "value": 0.55},
        "polarization": "unpolarized",
    },
    "compute": {"backend": "auto"},
}

print("Pixel config loaded: 2x2 BSI, 1.0 um pitch")

## 2. Define CRA vs Image Height Curve

The CRA (Chief Ray Angle) increases from the sensor center to the edge.
This curve depends on the camera lens design. Here we use a representative
curve typical of a mobile phone lens module.

Image height is normalized: 0.0 = center, 1.0 = corner.

In [ ]:
# CRA table: image_height -> CRA in degrees
# This represents a typical mobile lens with max CRA ~30 degrees at the corner
cra_table = [
    {"image_height": 0.0, "cra_deg": 0.0},
    {"image_height": 0.1, "cra_deg": 2.5},
    {"image_height": 0.2, "cra_deg": 5.2},
    {"image_height": 0.3, "cra_deg": 8.1},
    {"image_height": 0.4, "cra_deg": 11.5},
    {"image_height": 0.5, "cra_deg": 15.0},
    {"image_height": 0.6, "cra_deg": 18.5},
    {"image_height": 0.7, "cra_deg": 22.0},
    {"image_height": 0.8, "cra_deg": 25.2},
    {"image_height": 0.9, "cra_deg": 27.8},
    {"image_height": 1.0, "cra_deg": 30.0},
]

# Visualize the CRA curve
ih_vals = [entry["image_height"] for entry in cra_table]
cra_vals = [entry["cra_deg"] for entry in cra_table]

plt.figure(figsize=(8, 4))
plt.plot(ih_vals, cra_vals, "o-", linewidth=2, color="tab:blue")
plt.xlabel("Image Height (normalized)")
plt.ylabel("CRA (degrees)")
plt.title("Chief Ray Angle vs Image Height")
plt.grid(True, alpha=0.3)
plt.tight_layout()

## 3. Run ROI Sweep

The `ROISweepRunner` runs a simulation at each image height position.
At each position, it:
- Interpolates the CRA from the table
- Sets the source angle to match the CRA
- Applies automatic microlens shift to compensate
- Runs the simulation

In [ ]:
# Define sweep positions (sample at 6 image heights for speed)
image_heights = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]

roi_config = {
    "image_heights": image_heights,
    "cra_table": cra_table,
}

print(f"Running ROI sweep at {len(image_heights)} positions...")
results = ROISweepRunner.run(config, roi_config)

print(f"\nCompleted. Keys: {list(results.keys())}")

## 4. Extract QE per Channel at Each Position

For each image height, extract the average QE for each color channel
(R, G, B) from the simulation result.

In [ ]:
# Extract per-channel QE at each image height
channels = {"R": [], "G": [], "B": []}
avg_qe_all = []

for ih in image_heights:
    key = f"ih_{ih:.2f}"
    result = results[key]

    for ch in channels:
        ch_pixels = [
            float(np.mean(qe))
            for name, qe in result.qe_per_pixel.items()
            if name.startswith(ch)
        ]
        channels[ch].append(np.mean(ch_pixels) if ch_pixels else 0.0)

    all_qe = float(np.mean([np.mean(qe) for qe in result.qe_per_pixel.values()]))
    avg_qe_all.append(all_qe)

# Print summary table
print(f"{'IH':>6}  {'CRA':>6}  {'R QE':>8}  {'G QE':>8}  {'B QE':>8}  {'Avg':>8}")
print("-" * 52)
for i, ih in enumerate(image_heights):
    cra = np.interp(ih, ih_vals, cra_vals)
    print(f"{ih:6.2f}  {cra:5.1f}°  {channels['R'][i]:8.3f}  "
          f"{channels['G'][i]:8.3f}  {channels['B'][i]:8.3f}  "
          f"{avg_qe_all[i]:8.3f}")

## 5. Plot QE vs Image Height

Visualize how QE decreases toward the sensor edge due to increasing CRA,
showing each color channel separately.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

channel_colors = {"R": "red", "G": "green", "B": "blue"}

for ch, color in channel_colors.items():
    ax.plot(image_heights, channels[ch], "o-",
            color=color, label=f"{ch} channel", linewidth=2, markersize=8)

ax.plot(image_heights, avg_qe_all, "k--",
        label="All-channel average", linewidth=1.5, alpha=0.6)

ax.set_xlabel("Image Height (normalized)")
ax.set_ylabel("QE at 550 nm")
ax.set_title("Quantum Efficiency vs Sensor Position (with ML shift)")
ax.legend()
ax.set_ylim(0, 1)
ax.grid(True, alpha=0.3)

# Add secondary x-axis showing CRA
ax2 = ax.twiny()
cra_at_positions = [np.interp(ih, ih_vals, cra_vals) for ih in image_heights]
ax2.set_xlim(ax.get_xlim())
ax2.set_xticks(image_heights)
ax2.set_xticklabels([f"{c:.0f}°" for c in cra_at_positions])
ax2.set_xlabel("CRA (degrees)")

fig.tight_layout()

## 6. Relative Illumination Map

Relative illumination (RI) is the ratio of QE at each position to the center QE.
RI = 1.0 at center, decreasing toward the edge. Sensor designers typically
require RI > 0.5 at the sensor corner.

In [ ]:
# Compute relative illumination (normalized to center)
center_qe = avg_qe_all[0]  # image height = 0.0
ri_all = [qe / center_qe for qe in avg_qe_all]

ri_per_channel = {}
for ch in channels:
    center_ch_qe = channels[ch][0]
    if center_ch_qe > 0:
        ri_per_channel[ch] = [qe / center_ch_qe for qe in channels[ch]]
    else:
        ri_per_channel[ch] = [0.0] * len(image_heights)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Left: RI per channel
for ch, color in channel_colors.items():
    ax1.plot(image_heights, ri_per_channel[ch], "o-",
             color=color, label=f"{ch} channel", linewidth=2)
ax1.plot(image_heights, ri_all, "k--", label="Average", linewidth=1.5)
ax1.axhline(0.5, color="gray", linestyle=":", alpha=0.5, label="RI = 0.5 limit")
ax1.set_xlabel("Image Height (normalized)")
ax1.set_ylabel("Relative Illumination")
ax1.set_title("Relative Illumination vs Image Height")
ax1.legend(loc="lower left")
ax1.set_ylim(0, 1.1)
ax1.grid(True, alpha=0.3)

# Right: 2D RI map (radial symmetry assumed)
r = np.linspace(0, 1, 100)
ri_interp = np.interp(r, image_heights, ri_all)

theta_grid = np.linspace(0, 2 * np.pi, 100)
r_grid, theta_2d = np.meshgrid(r, theta_grid)
x_grid = r_grid * np.cos(theta_2d)
y_grid = r_grid * np.sin(theta_2d)

# Map RI onto the 2D grid
ri_2d = np.interp(r_grid, image_heights, ri_all)

im = ax2.pcolormesh(x_grid, y_grid, ri_2d, cmap="viridis", vmin=0.4, vmax=1.0)
ax2.set_aspect("equal")
ax2.set_title("Relative Illumination Map (sensor plane)")
ax2.set_xlabel("x (normalized)")
ax2.set_ylabel("y (normalized)")
plt.colorbar(im, ax=ax2, label="Relative Illumination")

fig.tight_layout()

## 7. Spectral ROI Sweep

For a more complete analysis, run a wavelength sweep at each ROI position.
This shows how spectral response changes across the sensor.

In [ ]:
# Switch to wavelength sweep mode
config_spectral = dict(config)
config_spectral["source"] = {
    "wavelength": {
        "mode": "sweep",
        "sweep": {"start": 0.40, "stop": 0.70, "step": 0.02},
    },
    "polarization": "unpolarized",
}

# Sweep at 3 representative positions for speed
roi_spectral = {
    "image_heights": [0.0, 0.5, 1.0],
    "cra_table": cra_table,
}

print("Running spectral ROI sweep (3 positions x 16 wavelengths)...")
spectral_results = ROISweepRunner.run(config_spectral, roi_spectral)
print("Done.")

In [ ]:
# Plot spectral QE at center, mid-field, and edge
fig, ax = plt.subplots(figsize=(10, 6))

positions = {"ih_0.00": ("Center (0.0)", "-"),
             "ih_0.50": ("Mid-field (0.5)", "--"),
             "ih_1.00": ("Edge (1.0)", ":")}

for key, (label, ls) in positions.items():
    result = spectral_results[key]
    wl_nm = result.wavelengths * 1000

    # Average across green pixels
    green_qe = np.mean([
        qe for name, qe in result.qe_per_pixel.items()
        if name.startswith("G")
    ], axis=0)

    ax.plot(wl_nm, green_qe, ls, label=f"Green - {label}", linewidth=2)

ax.set_xlabel("Wavelength (nm)")
ax.set_ylabel("QE")
ax.set_title("Green QE Spectrum at Different Sensor Positions")
ax.legend()
ax.set_ylim(0, 1)
ax.grid(True, alpha=0.3)
fig.tight_layout()

## Summary

This notebook demonstrated the ROI sweep workflow:

1. **CRA table** defines the lens-dependent angle variation across the sensor
2. **ROISweepRunner** automates the per-position simulation with CRA and microlens shift
3. **Per-channel QE** shows how each color channel responds to increasing CRA
4. **Relative illumination** quantifies the sensor-edge QE loss
5. **Spectral ROI sweep** reveals wavelength-dependent position sensitivity

Key findings:
- Blue channel degrades fastest at high CRA due to shallow absorption
- Microlens auto-shift partially compensates but cannot fully recover edge QE
- Relative illumination drops to 60-70% at the sensor corner for a typical mobile lens

Next: See `05_stability_demo.ipynb` for numerical stability analysis.